In [ ]:
# import libraries
import requests
import time
import datetime
import pandas as pd
import smtplib
from lxml import html
import matplotlib.pyplot as plt
import re
import json
from parsel import Selector

In [ ]:
# connect to website
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.48'}
base_url = 'http://ufcstats.com/statistics/events/completed?page=all'

# base_tree = html.fromstring(requests.get(base_url, headers=headers).content)
response = requests.get(base_url, headers=headers)
print(response)



In [ ]:
def get_events(response):
    selector = Selector(text = response.text)
    
    ufc_events = []
    for events in selector.css('tr.b-statistics__table-row'):
        event = {
            'name': None,
            'date': None,
            'location': None,
            'link': None,
        }

        try:
            event['name'] = events.css('a.b-link_style_black::text').get().strip()
            event['date'] = events.css('span.b-statistics__date::text').get().strip()
            event['location'] =  events.css('td.b-statistics__table-col_style_big-top-padding::text').get().strip()
            event['link'] =  events.css('a.b-link_style_black').attrib['href']

            ufc_events.append(event)

        except:
            pass

    return ufc_events

In [ ]:
events = get_events(response)
ufc_events = pd.DataFrame(events)


In [ ]:
def get_fights(response, headers, number_of_events: int = 1):
    events = get_events(response)
    n = number_of_events
    ufc_fights = []
    for event in events[:n]:
        url = event['link']
        event_name = event['name']
        selector = Selector(requests.get(url, headers=headers).text)

        for fights in selector.css('tr.b-fight-details__table-row')[1:]:

            fight = {
                'event_name': None,
                'fight_name': None,
                'weight_class': None,
                'method': None,
                'total_rounds': None,
                'time': None,
                'link': None
            }
            try:
                fight['event_name'] = event_name
                fighter1 = fights.css('td:nth-child(2) > p:nth-child(1) > a::text').get().strip()
                fighter2 = fights.css('td:nth-child(2) > p:nth-child(2) > a::text').get().strip()
                fight['fight_name'] = fighter1 + ' vs ' + fighter2
                fight['weight_class'] = fights.css('td:nth-child(7) > p::text').get().strip()
                fight['method'] = fights.css('td:nth-child(8) > p:nth-child(1)::text').get().strip()
                fight['total_rounds'] = fights.css('td:nth-child(9) > p::text').get().strip()
                fight['time'] = fights.css('td:nth-child(10) > p::text').get().strip()
                fight['link'] = fights.css('td > p > a').attrib['href']

                ufc_fights.append(fight)

            except:
                pass

    return ufc_fights

In [ ]:
events = get_events(response)
n = 20 + 1
for event in events[:n]:
    # print(event['name'])
    url = event['link']
    print(url)

In [ ]:
print(len(get_fights(response, headers, 5)))

In [140]:
def get_fights_records(response, headers, number_of_events: int = 1):
    fights = get_fights(response, headers, number_of_events)
    ufc_fight_records = []

    for fight in fights:
        url = fight['link']
        selector = Selector(requests.get(url, headers=headers).text)
        for rounds in range(int(fight['total_rounds'])):
            table_row = 2 * rounds + 2
            for records in selector.css('div.b-fight-details'):
                ufc_fight_records_A = {
                    'event_name': fight['event_name'],
                    'fight_name': fight['fight_name'],
                    'weight_class': fight['weight_class'],
                    'winning_method': fight['method'],
                    'total_rounds': fight['total_rounds'],
                    'time': fight['time'],
                    'link': fight['link'],
                    'fighter_name': None,
                    'opponent_name': None,
                    'win/loss': None,
                    'round': None,
                    'Knock_Down': None,
                    'Take_Down': None,
                    'Submission_Attempt': None,
                    'Reversal': None,
                    'Control_Time': None,
                    'Head': None,
                    'Body': None,
                    'Leg': None,
                    'Distance': None,
                    'Clinch': None,
                    'Ground': None,
                }

                ufc_fight_records_B = {
                    'event_name': fight['event_name'],
                    'fight_name': fight['fight_name'],
                    'weight_class': fight['weight_class'],
                    'winning_method': fight['method'],
                    'total_rounds': fight['total_rounds'],
                    'time': fight['time'],
                    'link': fight['link'],
                    'fighter_name': None,
                    'opponent_name': None,
                    'win/loss': None,
                    'round': None,
                    'Knock_Down': None,
                    'Take_Down': None,
                    'Submission_Attempt': None,
                    'Reversal': None,
                    'Control_Time': None,
                    'Head': None,
                    'Body': None,
                    'Leg': None,
                    'Distance': None,
                    'Clinch': None,
                    'Ground': None,
                }
                try:
                    ufc_fight_records_A['fighter_name'] = records.css('div.b-fight-details__persons.clearfix > div:nth-child(1) > div > h3 > a::text').get().strip()
                    ufc_fight_records_A['opponent_name'] = records.css('div.b-fight-details__persons.clearfix > div:nth-child(2) > div > h3 > a::text').get().strip()
                    ufc_fight_records_A['win/loss'] = records.css('div.b-fight-details__persons.clearfix > div:nth-child(1) > i::text').get().strip()
                    ufc_fight_records_A['round'] = rounds + 1
                    ufc_fight_records_A['Knock_Down'] = records.css(f'section:nth-child(5) > table > tbody > tr:nth-child({table_row})> td:nth-child(2) > p:nth-child(1)::text').get().strip()
                    ufc_fight_records_A['Take_Down'] = records.css(f'section:nth-child(5) > table > tbody > tr:nth-child({table_row})> td:nth-child(6) > p:nth-child(1)::text').get().strip()
                    ufc_fight_records_A['Submission_Attempt'] = records.css(f'section:nth-child(5) > table > tbody > tr:nth-child({table_row})> td:nth-child(8) > p:nth-child(1)::text').get().strip()
                    ufc_fight_records_A['Reversal'] = records.css(f'section:nth-child(5) > table > tbody > tr:nth-child({table_row})> td:nth-child(9) > p:nth-child(1)::text').get().strip()
                    ufc_fight_records_A['Control_Time'] = records.css(f'section:nth-child(5) > table > tbody > tr:nth-child({table_row})> td:nth-child(10) > p:nth-child(1)::text').get().strip()
                    ufc_fight_records_A['Head'] = records.css(f'section:nth-child(8) > table > tbody > tr:nth-child({table_row})> td:nth-child(4) > p:nth-child(1)::text').get().strip()
                    ufc_fight_records_A['Body'] = records.css(f'section:nth-child(8) > table > tbody > tr:nth-child({table_row})> td:nth-child(5) > p:nth-child(1)::text').get().strip()
                    ufc_fight_records_A['Leg'] = records.css(f'section:nth-child(8) > table > tbody > tr:nth-child({table_row})> td:nth-child(6) > p:nth-child(1)::text').get().strip()
                    ufc_fight_records_A['Distance'] = records.css(f'section:nth-child(8) > table > tbody > tr:nth-child({table_row})> td:nth-child(7) > p:nth-child(1)::text').get().strip()
                    ufc_fight_records_A['Clinch'] = records.css(f'section:nth-child(8) > table > tbody > tr:nth-child({table_row})> td:nth-child(8) > p:nth-child(1)::text').get().strip()
                    ufc_fight_records_A['Ground'] = records.css(f'section:nth-child(8) > table > tbody > tr:nth-child({table_row})> td:nth-child(9) > p:nth-child(1)::text').get().strip()

                    ufc_fight_records_B['fighter_name'] = records.css('div.b-fight-details__persons.clearfix > div:nth-child(2) > div > h3 > a::text').get().strip()
                    ufc_fight_records_B['opponent_name'] = records.css('div.b-fight-details__persons.clearfix > div:nth-child(1) > div > h3 > a::text').get().strip()
                    ufc_fight_records_B['win/loss'] = records.css('div.b-fight-details__persons.clearfix > div:nth-child(2) > i::text').get().strip()
                    ufc_fight_records_B['round'] = rounds + 1
                    ufc_fight_records_B['Knock_Down'] = records.css(f'section:nth-child(5) > table > tbody > tr:nth-child({table_row})> td:nth-child(2) > p:nth-child(2)::text').get().strip()
                    ufc_fight_records_B['Take_Down'] = records.css(f'section:nth-child(5) > table > tbody > tr:nth-child({table_row})> td:nth-child(6) > p:nth-child(2)::text').get().strip()
                    ufc_fight_records_B['Submission_Attempt'] = records.css(f'section:nth-child(5) > table > tbody > tr:nth-child({table_row})> td:nth-child(8) > p:nth-child(2)::text').get().strip()
                    ufc_fight_records_B['Reversal'] = records.css(f'section:nth-child(5) > table > tbody > tr:nth-child({table_row})> td:nth-child(9) > p:nth-child(2)::text').get().strip()
                    ufc_fight_records_B['Control_Time'] = records.css(f'section:nth-child(5) > table > tbody > tr:nth-child({table_row})> td:nth-child(10) > p:nth-child(2)::text').get().strip()
                    ufc_fight_records_B['Head'] = records.css(f'section:nth-child(8) > table > tbody > tr:nth-child({table_row})> td:nth-child(4) > p:nth-child(2)::text').get().strip()
                    ufc_fight_records_B['Body'] = records.css(f'section:nth-child(8) > table > tbody > tr:nth-child({table_row})> td:nth-child(5) > p:nth-child(2)::text').get().strip()
                    ufc_fight_records_B['Leg'] = records.css(f'section:nth-child(8) > table > tbody > tr:nth-child({table_row})> td:nth-child(6) > p:nth-child(2)::text').get().strip()
                    ufc_fight_records_B['Distance'] = records.css(f'section:nth-child(8) > table > tbody > tr:nth-child({table_row})> td:nth-child(7) > p:nth-child(2)::text').get().strip()
                    ufc_fight_records_B['Clinch'] = records.css(f'section:nth-child(8) > table > tbody > tr:nth-child({table_row})> td:nth-child(8) > p:nth-child(2)::text').get().strip()
                    ufc_fight_records_B['Ground'] = records.css(f'section:nth-child(8) > table > tbody > tr:nth-child({table_row})> td:nth-child(9) > p:nth-child(2)::text').get().strip()

                    ufc_fight_records.append(ufc_fight_records_A)
                    ufc_fight_records.append(ufc_fight_records_B)

                except:
                    pass

    return ufc_fight_records



In [ ]:
fights = get_fights(response, headers, 2)

for fight in fights:
    url = fight['link']
    selector = Selector(requests.get(url, headers=headers).text)

    for records in selector.css('div.b-fight-details'):
        ufc_fight_records_A = {
            'event_name': fight['event_name'],
            'fight_name': fight['fight_name'],
            'weight_class': fight['weight_class'],
            'winning_method': fight['method'],
            'total_rounds': fight['total_rounds'],
            'time': fight['time'],
            'link': fight['link'],
            'fighter_name': None,
            'opponent_name': None,
            'win/loss': None,
            'round': None,
            'Knock_Down': None,
            'Take_Down': None,
            'Submission_Attempt': None,
            'Reversal': None,
            'Control_Time': None,
            'Head': None,
            'Body': None,
            'Leg': None,
            'Distance': None,
            'Clinch': None,
            'Ground': None,
        }

        ufc_fight_records_B = {
            'event_name': fight['event_name'],
            'fight_name': fight['fight_name'],
            'weight_class': fight['weight_class'],
            'winning_method': fight['method'],
            'total_rounds': fight['total_rounds'],
            'time': fight['time'],
            'link': fight['link'],
            'fighter_name': None,
            'opponent_name': None,
            'win/loss': None,
            'round': None,
            'Knock_Down': None,
            'Take_Down': None,
            'Submission_Attempt': None,
            'Reversal': None,
            'Control_Time': None,
            'Head': None,
            'Body': None,
            'Leg': None,
            'Distance': None,
            'Clinch': None,
            'Ground': None,
        }

        try:
            for rounds in range(1, int(fight['total_rounds'])):
                print(rounds)
        #         table_row = 2 * rounds + 2
        #     ufc_fight_records_A['fighter_name'] = records.css('div.b-fight-details__persons.clearfix > div:nth-child(1) > div > h3 > a::text').get().strip()
        #     ufc_fight_records_A['opponent_name'] = records.css('div.b-fight-details__persons.clearfix > div:nth-child(2) > div > h3 > a::text').get().strip()
        #     ufc_fight_records_A['win/loss'] = records.css('div.b-fight-details__persons.clearfix > div:nth-child(1) > i::text').get().strip()
        #     # ufc_fight_records_A['round'] = rounds
        # KD = records.css('section:nth-child(5) > table > tbody > tr:nth-child(5) > td:nth-child(10) > p:nth-child(1)').get()
        # print(KD)

        #     ufc_fight_records_A['Take_Down'] = records.css(f'section:nth-child(5) > table > tbody:nth-child(4) > tr > td:nth-child(6) > p:nth-child(1)::text').get().strip()
        #     ufc_fight_records_A['Submission_Attempt'] = records.css(f'section:nth-child(5) > table > tbody:nth-child(4) > tr > td:nth-child(8) > p:nth-child(1)::text').get().strip()
        #     ufc_fight_records_A['Reversal'] = records.css(f'section:nth-child(5) > table > tbody:nth-child(4) > tr > td:nth-child(9) > p:nth-child(1)::text').get().strip()
        #     ufc_fight_records_A['Control_Time'] = records.css(f'section:nth-child(5) > table > tbody:nth-child(4) > tr > td:nth-child(10) > p:nth-child(1)::text').get().strip()
        #     # ufc_fight_records_A['Head'] = records.css(f'section:nth-child(8) > table > tbody:nth-child(4) > tr > td:nth-child(4) > p:nth-child(1)::text').get().strip()
        #     ufc_fight_records_A['Body'] = records.css(f'section:nth-child(8) > table > tbody:nth-child(4) > tr > td:nth-child(5) > p:nth-child(1)::text').get().strip()
        #     ufc_fight_records_A['Leg'] = records.css(f'section:nth-child(8) > table > tbody:nth-child(4) > tr > td:nth-child(6) > p:nth-child(1)::text').get().strip()
        #     ufc_fight_records_A['Distance'] = records.css(f'section:nth-child(8) > table > tbody:nth-child(4) > tr > td:nth-child(7) > p:nth-child(1)::text').get().strip()
        #     ufc_fight_records_A['Clinch'] = records.css(f'section:nth-child(8) > table > tbody:nth-child(4) > tr > td:nth-child(8) > p:nth-child(1)::text').get().strip()
        #     ufc_fight_records_A['Ground'] = records.css(f'section:nth-child(8) > table > tbody:nth-child(4) > tr > td:nth-child(9) > p:nth-child(1)::text').get().strip()

        #     ufc_fight_records_B['fighter_name'] = records.css('div.b-fight-details__persons.clearfix > div:nth-child(2) > div > h3 > a::text').get().strip()
        #     ufc_fight_records_B['opponent_name'] = records.css('div.b-fight-details__persons.clearfix > div:nth-child(1) > div > h3 > a::text').get().strip()
        #     ufc_fight_records_B['win/loss'] = records.css('div.b-fight-details__persons.clearfix > div:nth-child(2) > i::text').get().strip()
        #     # ufc_fight_records_B['round'] = rounds
        #     ufc_fight_records_B['Knock_Down'] = records.css('section:nth-child(5) > table > tbody:nth-child(4) > tr > td:nth-child(2) > p:nth-child(2)::text').get().strip()
        #     ufc_fight_records_B['Take_Down'] = records.css(f'section:nth-child(5) > table > tbody:nth-child(4) > tr > td:nth-child(6) > p:nth-child(2)::text').get().strip()
        #     ufc_fight_records_B['Submission_Attempt'] = records.css(f'section:nth-child(5) > table > tbody:nth-child(4) > tr > td:nth-child(8) > p:nth-child(2)::text').get().strip()
        #     ufc_fight_records_B['Reversal'] = records.css(f'section:nth-child(5) > table > tbody:nth-child(4) > tr > td:nth-child(9) > p:nth-child(2)::text').get().strip()
        #     ufc_fight_records_B['Control_Time'] = records.css(f'section:nth-child(5) > table > tbody:nth-child(4) > tr > td:nth-child(10) > p:nth-child(2)::text').get().strip()
        #     ufc_fight_records_B['Head'] = records.css(f'section:nth-child(8) > table > tbody:nth-child(4) > tr > td:nth-child(4) > p:nth-child(2)::text').get().strip()
        #     ufc_fight_records_B['Body'] = records.css(f'section:nth-child(8) > table > tbody:nth-child(4) > tr > td:nth-child(5) > p:nth-child(2)::text').get().strip()
        #     ufc_fight_records_B['Leg'] = records.css(f'section:nth-child(8) > table > tbody:nth-child(4) > tr > td:nth-child(6) > p:nth-child(2)::text').get().strip()
        #     ufc_fight_records_B['Distance'] = records.css(f'section:nth-child(8) > table > tbody:nth-child(4) > tr > td:nth-child(7) > p:nth-child(2)::text').get().strip()
        #     ufc_fight_records_B['Clinch'] = records.css(f'section:nth-child(8) > table > tbody:nth-child(4) > tr > td:nth-child(8) > p:nth-child(2)::text').get().strip()
        #     ufc_fight_records_B['Ground'] = records.css(f'section:nth-child(8) > table > tbody:nth-child(4) > tr > td:nth-child(9) > p:nth-child(2)::text').get().strip()

        #     ufc_fight_records.append(ufc_fight_records_A)
        #     ufc_fight_records.append(ufc_fight_records_B)

        except:
            pass





In [141]:
ufc_fights_records_df = pd.DataFrame(get_fights_records(response, headers, 1))

In [129]:
print(ufc_fights_records_df.head())

                        event_name                  fight_name  weight_class  \
0  UFC Fight Night: Song vs. Simon  Song Yadong vs Ricky Simon  Bantamweight   
1  UFC Fight Night: Song vs. Simon  Song Yadong vs Ricky Simon  Bantamweight   
2  UFC Fight Night: Song vs. Simon  Song Yadong vs Ricky Simon  Bantamweight   
3  UFC Fight Night: Song vs. Simon  Song Yadong vs Ricky Simon  Bantamweight   
4  UFC Fight Night: Song vs. Simon  Song Yadong vs Ricky Simon  Bantamweight   

  winning_method total_rounds  time  \
0         KO/TKO            5  1:10   
1         KO/TKO            5  1:10   
2         KO/TKO            5  1:10   
3         KO/TKO            5  1:10   
4         KO/TKO            5  1:10   

                                                link fighter_name  \
0  http://ufcstats.com/fight-details/2b1ac5d8f986...  Song Yadong   
1  http://ufcstats.com/fight-details/2b1ac5d8f986...  Ricky Simon   
2  http://ufcstats.com/fight-details/2b1ac5d8f986...  Song Yadong   
3  http:

In [142]:
ufc_fights_records_df.to_csv('ufc_fights_records.csv', index=False)